# PhysioNet/Computing in Cardiology Challenge 2020
## Classification of 12-lead ECGs
### 2. Create Training Dataset

# Setup Notebook

In [7]:
# Import 3rd party libraries
import os
import sys
import time
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Import local Libraries
sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd()))))))
from kardioml import DATA_PATH, ECG_LEADS, FS, LABELS_LOOKUP 

# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get List of files

In [8]:
# Get meta data files
filenames = [file.split('.')[0] for file in os.listdir(os.path.join(DATA_PATH, 'formatted')) if 'json' in file]

# Process Meta Data

In [9]:
# Create directory for formatted data
os.makedirs(os.path.join(DATA_PATH, 'physionet_2017', 'training'), exist_ok=True)

# List of meta data
meta_data = list()

# Open files and extract meta data
for filename in filenames:
    
    # Import meta data
    data = json.load(open(os.path.join(DATA_PATH, 'formatted', '{}.json'.format(filename))))
    meta_data.append(data)
    
# Generate DataFrame
meta_data = pd.DataFrame(meta_data)

# Set numeric
meta_data['age'] = meta_data['age'].apply(pd.to_numeric, errors='coerce')

# Add columns
meta_data['label_count'] = meta_data['labels'].map(lambda val: len(val))
meta_data['length'] = meta_data['shape'].map(lambda val: val[1] * 1 / FS)
meta_data['label_count'] = meta_data['labels'].map(lambda val: len(val))
meta_data['labels_concat'] = meta_data['labels_full'].map(lambda val: '{}\n{}'.format(val[0], val[1]) 
                                                          if len(val) == 2 else val[0])

# Save 
meta_data.to_csv(os.path.join(DATA_PATH, 'physionet_2017', 'training', 'meta_data.csv'), index=False)
    
# View DataFrame
meta_data.head()

,age,channel_order,filename,label_train,labels,labels_full,labels_int,sex,shape,label_count,length,labels_concat
0,74.0,"[I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5...",A0001,"[0, 0, 0, 0, 0, 0, 1, 0, 0]",[RBBB],[Right bundle branch block],[6],Male,"[12, 7500]",1,15.000,Right bundle branch block
1,49.0,"[I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5...",A0002,"[0, 0, 0, 1, 0, 0, 0, 0, 0]",[Normal],[Normal sinus rhythm],[3],Female,"[12, 5000]",1,10.000,Normal sinus rhythm
2,81.0,"[I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5...",A0003,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",[AF],[Atrial fibrillation],[0],Female,"[12, 5000]",1,10.000,Atrial fibrillation
3,45.0,"[I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5...",A0004,"[1, 0, 0, 0, 0, 0, 0, 0, 0]",[AF],[Atrial fibrillation],[0],Male,"[12, 5974]",1,11.948,Atrial fibrillation
4,53.0,"[I, II, III, aVR, aVL, aVF, V1, V2, V3, V4, V5...",A0005,"[0, 0, 0, 0, 0, 1, 0, 0, 0]",[PVC],[Premature ventricular complex],[5],Male,"[12, 12500]",1,25.000,Premature ventricular complex


# Process Labels

In [10]:
# Get labels
labels = pd.DataFrame(data=np.asarray(meta_data['label_train'].values.tolist(),  dtype=np.int32), 
                      columns=list(LABELS_LOOKUP.keys()))

# Save 
labels.to_csv(os.path.join(DATA_PATH, 'physionet_2017', 'training', 'labels.csv'), index=False)

# View DataFrame
labels.head()

,AF,I-AVB,LBBB,Normal,PAC,PVC,RBBB,STD,STE
0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0


# Process Features

In [11]:
# Set lead
lead = 'I'

# List of meta data
features = list()

# Open files and extract meta data
for filename in filenames:
    
    # Import features
    data_features = json.load(open(os.path.join(DATA_PATH, 'physionet_2017', 'features', lead, '{}.json'.format(filename))))
    
    # Import meta data
    data_meta = json.load(open(os.path.join(DATA_PATH, 'formatted', '{}.json'.format(filename))))
    
    features.append(data_features)
    
# Generate DataFrame
features = pd.DataFrame(features)

# Binarize Sex label
label_encoder = LabelEncoder()
features['sex'] = label_encoder.fit_transform(features['sex'])

# Save 
features.to_csv(os.path.join(DATA_PATH, 'physionet_2017', 'training', 'features.csv'), index=False)
    
# View DataFrame
features.head()

,full_waveform_min,full_waveform_max,full_waveform_mean,full_waveform_median,full_waveform_std,full_waveform_skew,full_waveform_kurtosis,full_waveform_duration,swt_d_1_low_power_ratio,swt_d_1_med_power_ratio,...,rpeak_entropy,rpeak_higuchi_fractal_dimension,template_corr_coeff_mean,template_corr_coeff_std,qrs_corr_coeff_mean,qrs_corr_coeff_std,p_wave_corr_coeff_mean,p_wave_corr_coeff_std,t_wave_corr_coeff_mean,t_wave_corr_coeff_std
0,-0.976816,1.061664,0.000247,-0.001915,0.267624,0.035163,4.168482,14.996,0.423375,0.113496,...,3.317816,2.465088,0.987405,0.003816,0.995713,0.002541,0.866373,0.067781,0.942713,0.022492
1,-0.625051,1.044477,-0.000199,-0.005681,0.167347,2.358342,15.457311,9.996,0.600130,0.098219,...,3.091042,NaN,0.979281,0.006906,0.990972,0.006404,0.886879,0.045675,0.688547,0.088916
2,-0.360254,1.081835,0.000477,-0.025866,0.196894,3.331854,12.586657,9.996,0.565341,0.085480,...,3.135494,2.286132,0.672160,0.178956,0.984642,0.007182,0.471329,0.307302,0.060507,0.327653
3,-0.595681,1.099412,0.000537,-0.015885,0.208745,2.669416,9.934822,11.944,0.601490,0.095649,...,2.877468,2.719416,0.849063,0.125237,0.977497,0.011820,0.376888,0.213236,0.121306,0.238374
4,-0.972058,1.315590,-0.000117,-0.014703,0.252693,1.051541,5.625249,24.996,0.682703,0.091991,...,3.020868,2.183621,0.840088,0.076720,0.946327,0.055893,0.433917,0.288018,0.430445,0.169273


In [12]:
# Data check
print(meta_data.shape)
print(labels.shape)
print(features.shape)

(6877, 12)
(6877, 9)
(6877, 155)
